## Initialisation

In [ ]:
import sqdtoolz as stz
import numpy as np

The first step is always to define a Laboratory object (see [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Laboratory.md) for more). The instrument config file `instr_config_file` is in YAML format, containing specifications for the instruments you will connect to. Examples of YAML entries are available in the documentation [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/InstrumentSpecific/Readme.md).

In [ ]:
lab = stz.Laboratory(instr_config_file = "your_config.yaml", save_dir = "Z:\\Data\\your_data_folder\\", using_VS_Code=True)

If the notebook crashes or is otherwise restarted, you can reload the Laboratory configuration from a previous experiment. This is useful for resetting instruments and local variables, to continue where you left off. If no directory is provided, it will reload the last completed experiment by default.

In [ ]:
lab.cold_reload_last_configuration(r"Z:\Data\your_data_folder\date\experiment\")

The lab browser is a convenient tool to visualise the current state of all instantiated instruments. To load, use `lab.open_browser()`

In [ ]:
lab.open_browser()

## Resonator spectroscopy by VNA

A vector network analyser (VNA) is useful for performing fast spectroscopy measurements with minimal set-up. It is able to output a wide-range sweep of frequencies through one port, and measure transmission through another port. To prepare resonator spectroscopy by VNA, connect the readout I/O lines to two different VNA ports. If the VNA has not been set up on a PC previously, some instructions exist on [Nuclino](https://app.nuclino.com/SQD-Lab/Admin/Portable-Keysight-VNA-64ab1677-395b-40c1-9154-45e2921c8be9).

The VNA can then be loaded according to its YAML entry in the loaded instrument config file. Here I assume that the reference key is `vna_ref`.

In [ ]:
# Vector network analyser - used for faster spectroscopy measurements
lab.load_instrument('vna_ref')
stz.ACQvna('VNA', lab, 'vna_ref')

The first line in the above verifies the device connectivity and loads the device. The second line instantiates a HAL object for the device, which can now be referred to as `lab.HAL('acq')`. Now that we have access to the VNA, let's configure the parameters to set up our experiment. An example is below.

In [ ]:
# Which ports are we using for input and output?
input_port = 1
output_port = 2

# Spectroscopy requires defining a range of frequencies over which to sweep.
freq_start = 6e9 # Hz
freq_end = 8e9 # Hz
sweep_type = 'Linear'
sweep_points = 1001

# VNA operation needs a signal power and measurement bandwidth. These can be tuned for optimal measurement.
vna_power = -30 # dBm
vna_bandwidth = 500 # Hz



In [ ]:
# Set parameters accordingly
lab.HAL('VNA').FrequencyStart = freq_start
lab.HAL('VNA').FrequencyEnd = freq_end
lab.HAL('VNA').SweepMode = sweep_type
lab.HAL('VNA').SweepPoints = sweep_points
lab.HAL('VNA').Power = vna_power
lab.HAL('VNA').Bandwidth = vna_bandwidth

Now that the instrument parameters are set, we want to save this particular configuration. This is useful for two reasons; (a) we can generate an ExperimentConfiguration object which we can use to perform experiments with these settings at any time, and (b) we can save the current ExperimentConfiguration for future reference, along with the generated experimental data. This is documented [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Exp_Config_Basic.md).

Below, an ExperimentConfiguration object with the current settings is instantiated. The duration is zero for now, as it is handled by the VNA - later we will see why this argument is useful. We are saving the settings of the VNA, and acquiring data with the VNA.

In [ ]:
experiment_name = 'vna_sweep'

vna_spec_config = stz.ExperimentConfiguration(name = experiment_name, lab = lab, 
                                            duration = 0, list_HALs = ['VNA'], hal_ACQ = 'VNA')

We can now perform a resonator spectroscopy experiment. Documentation on the experiment run functionality is available [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Exp_Overview.md).

In [ ]:
vna_sweep = stz.Experiment(name = 'vna_sweep', expt_config = lab.CONFIG('vna_sweep'))
data = lab.run_single(expt_obj = vna_sweep)

The data can now be visualised in `SQDViz` ([repo](https://github.com/sqdlab/SQDViz)), or the data itself can be accessed via `data.get_numpy_array()` and processed manually (as documented [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Data_IO.md)). If the resonator frequency is within the specified sweep, there should be a peak/trough at the appropriate frequency.

### Power sweep by VNA

It's often a good idea to perform a 2D sweep over signal frequency and signal power. In this way, we can see how the resonator peak/trough evolves with power. At sufficiently high powers, the regime of dispersive measurement breaks down and the spectroscopy resolution will worsen, before the peak/trough shifts in frequency and becomes highly resolved (a so-called 'bright state'). We want to stay at a power lower than the onset of these behaviours, but as high as possible to maximise signal-to-noise ratio.

In [ ]:
power_sweep = sqdt.Experiment('power_sweep', lab.CONFIG('vna_sweep'))

We need to define a variable which controls the power of the VNA signal. This is done with the `stz.VariableProperty` functionality. `Variable`s are well documented [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Var_Defns.md).

In [ ]:
stz.VariableProperty(name = 'vnaPower', lab = lab, sqdtoolz_obj = lab.HAL('VNA'), prop_name = 'Power')

start_power = -45
end_power = -15
step_size = 1

data = lab.run_single(power_sweep, [(lab.VAR('vnaPower'), (np.arange(start_power, end_power, step_size)))])

Again, the data can now be visualised in `SQDViz`, or the data itself can be accessed via `data.get_numpy_array()` and processed manually.

### Flux sweep by VNA

If the qubits are flux-tunable, we can see modulation of the resonators by tuning magnetic flux. This is an example procedure for using a superconducting magnetic coil, connected externally to Channel 1 of a programmable voltage source denoted in the config YAML by `v_source_ref`. First, the loading procedure for the voltage source.

In [ ]:
lab.load_instrument('v_source_ref')
stz.GENvoltSource('v_coil', lab, ['v_source_ref', 'CH1'])

The new ExperimentConfiguration, Experiment, VariableProperty, and execution will follow, as per the power sweep example.

In [ ]:
stz.ExperimentConfiguration('vna_flux_sweep', lab, 0, ['VNA', 'v_coil'], 'VNA')

vna_flux_sweep = sqdt.Experiment('vna_flux_sweep', lab.CONFIG('vna_flux_sweep'))

stz.VariableProperty('vFlux', lab, lab.HAL('v_coil'), 'Voltage')

In [ ]:
start_flux = -1
end_flux = 1
flux_step_size = 0.1

data = lab.run_single(vna_flux_sweep, [(lab.VAR('vFlux'), np.arange(start_flux, end_flux, flux_step_size))])

## Spectroscopy with microwave sources

### Resonator spectroscopy

After VNA spectroscopy provides a quick and dirty way to determine things like resonator frequencies and flux dependence, the use of microwave sources and dedicated acquisition devices permits more complex experiments. However, the procedure should start again with spectroscopy-type measurements. This time, the measured features (resonator frequency, optimal measurement power etc.) will be recorded for later use.

First we need to load an acquisition device. This could be an FPGA, the Tabor acquisition device, or the digitiser on the GPU PC. This is necessary to record our output measurement signal.

In [ ]:
lab.load_instrument('acq_device_ref')
stz.ACQ('acq', lab, 'acq_device_ref')

Next, we can load some microwave sources. First will be a source used for resonator spectroscopy. In addition, the acquisition device loaded above has some sample rate constraint which, for all our devices, prohibits accurately sampling microwave signals. So, we use a second microwave signal for downconversion purposes (see ... for an explanation). Finally, we will load a third microwave source for driving qubit/s for qubit spectroscopy. Documentation on microwave sources is available [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/GENmwSource.md).

In [ ]:
lab.load_instrument('res_source_ref')
stz.GENmwSource('MW_res', lab, 'res_source_ref', 'RFOUT')

lab.load_instrument('dnc_source_ref')
stz.GENmwSource('MW_dnc', lab, 'dnc_source_ref', 'RFOUT')

lab.load_instrument('qubit_source_ref')
stz.GENmwSource('MW_qubit', lab, 'qubit_source_ref', 'RFOUT')

Finally, we need an external synchronisation tool for timing of experiment repetitions. A DDG (digital delay generator) can be used to send trigger signals to other instruments, signalling the beginning of an experiment.

In [ ]:
# DDG (digital delay generator) - used to send trigger signals to other instruments, to begin an experiment
lab.load_instrument('ddg_ref')
stz.DDG('DDG', lab, 'ddg_ref')

We select an experimental repetition time which will contain the operations we wish to perform. Then, we prepare the DDG accordingly.

In [ ]:
repetition_time = 25e6
lab.HAL("DDG").RepetitionTime = repetition_time

# some settings for the DDG trigger
lab.HAL('DDG').get_trigger_output('AB').TrigPulseLength = 50e-9
lab.HAL('DDG').get_trigger_output('AB').TrigPolarity = 1
lab.HAL('DDG').get_trigger_output('AB').TrigPulseDelay = 0.0e-9
lab.HAL('DDG').get_trigger_output('AB').TrigEnable = True 

A typical set up is to feed the DDG signal (from Channel 'AB' above) into the trigger port of an AWG (arbitrary wave generator). The timing of subsequent signals in the experiment will then be controlled by the AWG. So, let's load the AWG. We need two channel outputs (for I and Q components) and a sample rate which is permitted by the relevant device.

In [ ]:
awg_sample_rate = 100e6

lab.load_instrument('awg_ref')
stz.WaveformAWG("WFM", lab, awg_channel_tuples=[('awg_ref', 'CH1'), ('awg_ref', 'CH2')], 
                            sample_rate=awg_sample_rate)

The total time of a waveform should be less than the total experimental repetition time.

In [ ]:
lab.HAL('WFM').set_valid_total_time(lab.HAL('DDG').RepetitionTime-1e-6)

Now let's set up the waveform on the AWG. Resonator spectroscopy is simply a long continuous resonator signal, so we need no AWG output -- but we are using it for timing purposes.

In [ ]:
# add a short zero-amplitude constant pulse
lab.HAL("WFM").add_waveform_segment(stz.WFS_Constant(name = "pulse", transform_func = None, time_len = 100e-9, value = 0.0))

# add another which persists for the rest of the waveform time (time_len = -1)
lab.HAL("WFM").add_waveform_segment(stz.WFS_Constant("zeros", None, -1, 0.0))

# configure the awg to output a marker signal when the 'pulse' component is on.
lab.HAL("WFM").get_output_channel(0).marker(0).set_markers_to_segments(["pulse"])

Now, the set up is as follows. The AWG begins an experiment when it receives a trigger from the DDG. This will commence the AWG waveform, which outputs a zero-amplitude pulse from both CH1 and CH2, and a marker pulse of length 100e-9 from the marker output. We will use this marker pulse to tell the acquisition to begin acquisition - so, we need to run a cable from the marker port of the AWG to the trigger port of the ACQ device.

We can produce timing diagrams based on the set-ups we design here (this will be shown later). To ensure these are accurate, we can mimic the physical trigger-marker set-ups we are configuring with the following functions:

In [ ]:
lab.HAL("WFM").set_trigger_source_all(lab.HAL("DDG").get_trigger_output('AB'))
lab.HAL("acq").set_trigger_source(lab.HAL("WFM").get_output_channel(0).marker(1))

Now that the acquisition device is being triggered, we need to configure how many samples it will measure, and how many repetitions the experiment will continue for. In the below example, we will configure $2^8$ repetitions of the experiment, with $2^{16}$ samples per experiment. The total measurement time can be calculated by NumSamples / SampleRate; this needs to be less than the experiment repetition time.

In [ ]:
total_time = lab.HAL('WFM').Duration
num_meas_samples = 2**11
meas_time = num_meas_samples / lab.HAL('acq').SampleRate
if meas_time < total_time:
    lab.HAL('acq').set_acq_params(reps = 2**8, segs = 1, samples = num_meas_samples)
else:
    print(f'{num_meas_samples} gives a total measurement time of {meas_time}, which is too long for a waveform of length {total_time}.')

The physical measurement set-up (with downconversion) means that each experimental repetition is measuring a 25 MHz signal. We process this signal digitally with the following steps: a digital downconversion to produce a DC signal, an FIR filter to reject undesirably frequency components, decimation to reduce the size of the dataset, and integration over the total number of samples measured. This results in a singular measurement (I,Q) which incorporates the amplitude and phase of the incoming signal. 

We encode this processing with a DataProcessor object. There are three variants; ProcessorFPGA, ProcessorGPU, and ProcessorCPU. Here I use ProcessorFPGA, for which some docs exist [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Proc_FPGA_Overview.md).

In [ ]:
myProc = stz.ProcessorFPGA(proc_name = 'fpga_dsp', lab = lab)
myProc.reset_pipeline()

myProc.add_stage(sqdt.FPGA_DDC([[25e6]]))
myProc.add_stage(sqdt.FPGA_FIR([[{'Type' : 'low', 'fc' : 10e6, 'Taps' : 40, 'Win' : 'hamming'}]]))
myProc.add_stage(sqdt.FPGA_Decimation('sample', 4)) 
myProc.add_stage(sqdt.FPGA_Integrate('segment'))
myProc.add_stage(sqdt.FPGA_Integrate('sample'))
myProc.add_stage(sqdt.FPGA_Mean('repetition'))

lab.HAL('acq').set_data_processor(myProc)

Finally, we can simply ensure that our resonator and downconversion microwave sources are turned on, and the qubit source is turned off.

In [ ]:
resonator_drive_power = -40

lab.HAL("MW_res").Mode = "Continuous" 
lab.HAL("MW_res").Power = resonator_drive_power
lab.HAL("MW_res").Output = True

lab.HAL("MW_dnc").Mode = "Continuous" 
lab.HAL("MW_dnc").Power = 16 
lab.HAL("MW_dnc").Output = True 

lab.HAL('MW_qubit').Output = False

With this, our experiment is prepared to run. We are going to save the current configuration of all our instruments into an ExperimentConfiguration; this is the object we pass to an experiment to say "set up the instruments as follows". It will also get saved alongside our data.

In [ ]:
stz.ExperimentConfiguration(name = 'contMeas', lab = lab, duration = repetition_time, 
                        list_HALs = ['DDG', 'WFM', 'MW_res', 'MW_dnc'], hal_ACQ = 'acq')

We can also view our experimental configuration as a timing diagram, for a sanity check.

In [ ]:
lab.CONFIG('contMeas').plot()

Running an experiment is then quite simple. We follow the same process as we did for the VNA.

In [ ]:
res_spec = Experiment('res_spec', lab.CONFIG('contMeas'))

stz.VariableProperty(name = 'res_freq', lab = lab, sqdtoolz_obj = lab.HAL('MW_res'), prop_name = 'Frequency')

freq_start = 6e9
freq_end = 8e9
step_size = 1e6

data = lab.run_single(res_spec, [(lab.VAR('res_freq'), (np.arange(freq_start, freq_end, step_size)))])

As for the VNA experiment, the data can now be visualised in `SQDViz`, or the data itself can be accessed via `data.get_numpy_array()` and processed manually. If the resonator frequency is within the specified sweep, there should be a peak/trough at the appropriate frequency.

However, there is also a pre-defined automated experiment which can do this work for us. The experiment `ExpPeakScouterIQ` will search for a Lorentzian shape in the measured data, fit this shape, and then yield the estimated resonator frequency. In the following cell, we define an `ExperimentSpecification` object for our resonator, which is used to store key information about the resonator, and then run an `ExpPeakScouterIQ` experiment.

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpPeakScouterIQ import *

stz.ExperimentSpecification(name = 'my_res', lab = lab, init_specs = 'Cavity')

fitted_res_spec = ExpPeakScouterIQ('fitted_res_spec', lab.CONFIG('contMeas'), 
                    param_centre=lab.SPEC('my_res')['Frequency'], param_width=lab.SPEC('my_res')['LineWidth'], 
                    is_trough=True, fit_fano_res=True)

fitted_res_spec_data = lab.run_single(fitted_res_spec, 
                    [(lab.VAR('res_freq'), (np.arange(freq_start, freq_end, step_size)))])

print(f'Frequency:')
print(lab.SPEC('my_res')['Frequency'].Value/1e9)

This fits the measured resonator spec, and saves the fitted resonant frequency to `lab.SPEC('my_res')['Frequency']`. A convenient feature of `lab.SPEC` objects is that they can be passed to experiments as configuration settings. For example, we can say to an experiment "use the frequency saved in `lab.SPEC('my_res')` to set the readout frequency". To assign a SPEC feature to a physical setting, we use the following line.

In [ ]:
lab.SPEC('my_res').set_destination('Frequency', lab.VAR('res_freq'))

The general procedure for resonator spectroscopy with microwave sources is complete. Below are generic procedures for extending this to power and flux sweeps.

In [ ]:
res_power_sweep = sqdt.Experiment('res_power_sweep', lab.CONFIG('res_spec'))

start_power = -45
end_power = -15
step_power = 1

stz.VariableProperty('res_power', lab, lab.HAL('MW_res'), 'Power')

power_sweep_data = lab.run_single(exp_power_sweep, 
                [(lab.VAR('res_power'), np.arange(start_power, end_power, step_power)), 
                (lab.VAR('res_freq'), np.arange(freq_start, freq_end, step_size))]) 

In [ ]:
# Select and save some sensible resonator power according to the above power sweep
lab.SPEC('my_res')['Power'] = -35
lab.SPEC('my_res').set_destination('Power', lab.VAR('res_power'))


In [ ]:
lab.CONFIG('contMeas').init_instruments() # used to initialise instruments according to said config
stz.ExperimentConfiguration(name = 'res_flux_sweep', lab = lab, duration = repetition_time, 
                        list_HALs = ['DDG', 'WFM', 'MW_res', 'MW_dnc', 'v_coil'], hal_ACQ = 'acq')

res_flux_sweep = sqdt.Experiment('res_flux_sweep', lab.CONFIG('res_flux_sweep'))

start_flux = -1
end_flux = 1
step_flux = 0.1

flux_sweep_data = lab.run_single(res_flux_sweep, 
                [(lab.VAR('vFlux'), np.arange(start_flux, end_flux, step_flux)), 
                (lab.VAR('res_freq'), np.arange(freq_start, freq_end, step_size))]) 

# reset flux to zero
lab.HAL('v_coil').Voltage = 0

### Qubit spectroscopy

In qubit spectroscopy, we want to fix the readout signal at the resonator frequency, turn on the qubit probe, and sweep the qubit probe frequency. So, let's initialise the instruments as per resonator spectroscopy, change the qubit source settings, and pass `lab.SPEC('my_res')` to the experiment.

In [ ]:
lab.CONFIG('contMeas').init_instruments()

lab.HAL('MW_qubit').Output = True
lab.HAL('MW_qubit').Mode = "Continuous" 
lab.HAL('MW_qubit').Power = -30

stz.ExperimentConfiguration('qubit_spec', lab, repetition_time, ['DDG','WFM','MW_res','MW_dnc','MW_qubit'], 
                            'acq', list_spec_names=['my_res'])

We can define an ExperimentSpecification for our qubit, and use ExpPeakScouterIQ to fit the qubit frequency. Typically, this experiment will start with a broad search for the qubit frequency, and be refined to get an increasingly better fit.

In [ ]:
stz.ExperimentSpecification(name = 'my_qubit', lab = lab, init_specs = 'Qubit')

stz.VariableProperty('qb_freq', lab, lab.HAL('MW_qubit'), 'Frequency')
stz.VariableProperty('qb_power', lab, lab.HAL('MW_qubit'), 'Power')

lab.SPEC('my_qubit').set_destination('Frequency GE', lab.VAR('qb_freq'))
lab.SPEC('my_qubit').set_destination('Power', lab.VAR('qb_power'))

exp_qubit_sweep = ExpPeakScouterIQ('qubit_spec', lab.CONFIG('qubit_spec'), 
                param_centre=lab.SPEC('my_qubit')['Frequency GE'], is_trough=False)

freq_start = 4e9
freq_end = 6e9
step_size = 3e6

data = lab.run_single(exp_qubit_sweep, [(lab.VAR('qb_freq'), np.arange(freq_start, freq_end, step_size))])

It can be useful to perform a qubit power sweep, to determine an optimal power for balanced between reduced linewidth and signal-to-noise ratio.

In [ ]:
qubit_power_sweep = sqdt.Experiment('qubit_power_sweep', lab.CONFIG('qubit_spec'))

start_power = -45
end_power = -15
step_power = 1

qb_power_sweep_data = lab.run_single(qubit_power_sweep, 
                [(lab.VAR('qb_power'), np.arange(start_power, end_power, step_power)), 
                (lab.VAR('res_freq'), np.arange(freq_start, freq_end, step_size))])

In [ ]:
# Select and save some sensible resonator power according to the above power sweep
lab.SPEC('my_qubit')['Power'] = -25

A flux sweep is slightly more involved. You must consider that the resonator peak will also move as flux is varied. Thus, we need to redo a resonator spectroscopy experiment before each qubit spectroscopy (i.e. at each new flux). Fortunately, `sqdtoolz` includes so-called 'grouped experiments', as documented [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Exp_CascadeGroup.md).

In [ ]:
lab.CONFIG('contMeas').init_instruments()

stz.ExperimentConfiguration('res_calibration', lab, repetition_time, 
                         ['DDG', 'WFM', 'MW_res', 'MW_dnc'], 'acq', list_spec_names=['my_res'])

res_calibration = ExpPeakScouterIQ('res_calibration', lab.CONFIG('res_calibration'), param_centre = lab.SPEC('my_res')['Frequency'], 
                    param_width=lab.SPEC('my_res')['LineWidth'], is_trough=True, fit_fano_res=True)

lab.CONFIG('qubit_spec').init_instruments()

stz.ExperimentConfiguration('qubit_flux_sweep', lab, repetition_time, ['DDG','WFM','MW_res','MW_dnc','MW_qubit', 'v_coil'], 
                            'acq', list_spec_names=['my_res', 'my_qubit'])

qubit_flux_sweep= Experiment('qubit_flux_sweep', lab.CONFIG('qubit_flux_sweep'))

start_flux = -1
end_flux = 1
flux_step_size = 0.1

lab.group_open("qubit_flux_exp")
for fluxV in lab.VAR('vFlux').arange(start_flux, end_flux, step_size):
    cavSpecData = lab.run_single(res_calibration, [(lab.VAR('res_freq'), np.arange(6e9, 8e9, 1e6))]) 
    cavSpecData.release()
    leData = lab.run_single(qubit_flux_sweep, [(lab.VAR('qubit_freq'), np.arange(4e9,5e9,2e6))]) 
    leData.release()
lab.group_close()

This experiment will be useful for determining/confirming the sweet-spot flux point and corresponding qubit frequency.

## Mixer calibration

Having identified the resonant frequencies of the resonator and qubit, we now want to transition to pulsed, time-domain control. To do this, we combine arbitrary pulse envelopes (generated by the AWG) with our continuous microwave signals. This is done with an IQ mixer; two channels of the AWG go into the I and Q ports respectively, the microwave source goes into the LO port, and the output is obtained at the RF port. Documentation for this process exists [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Mixer_Calibration.md). It is important to note that for mixer calibration, the acquisition device should be set up to acquire the mixed signal directly; we are no longer travelling through the quantum chip or fridge at all.

As seen in the docs linked above, mixer calibration relies on tuning four parameters; two external voltages which offset the I and Q AWG signals (`DCoff_I` and `DCoff_Q`), an amplitude factor which scales the two AWG signals (`AmpFac`) and a phase offset which shifts the default $\pi/2$ phase difference between the signals (`PhsOff`). Below we define each of these variables, and then store them in an ExperimentSpecification object which we can pass to future experiments in order to operate the mixer in the correct regime.

In [ ]:
stz.GENvoltSource('DCoff_I', lab, ['v_source_ref', 'CH2']) #v_source_ref was loaded earlier
stz.GENvoltSource('DCoff_Q', lab, ['v_source_ref', 'CH3'])

stz.WFMT_ModulationIQ('QubitFreqGE', lab, 100e6)

stz.ExperimentSpecification('MixerCalib', lab)
lab.SPEC('MixerCalib').add('IdcOff', 0, lab.HAL('DCoff_I'), 'Voltage')
lab.SPEC('MixerCalib').add('QdcOff', 0, lab.HAL('DCoff_Q'), 'Voltage')
lab.SPEC('MixerCalib').add('AmpFac', 0, lab.WFMT('QubitFreqGE'), 'IQAmplitudeFactor')
lab.SPEC('MixerCalib').add('PhsOff', 0, lab.WFMT('QubitFreqGE'), 'IQPhaseOffset')

Notice the introduction of the WaveformTransformation `QubitFreqGE`. This is an object which stores and later passes information about the appropriate scaled and phase-shifted envelopes to the AWG. This object is thoroughly documented [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/AWG_WFMTs.md).

An uncalibrated mixer yields a signal with three frequency components (a central LO component and two equidistant sidebands), of which we only wish to keep one sideband. First, we want to see these three peaks, in the uncalibrated regime.

We will initialise our instruments according to `contMeas`, but will have to make substantial changes for this measurement. This includes turning off the measurement signal (we are acquiring the qubit signal for mixer calibration), and turning up the power of the qubit source.

In [ ]:
lab.CONFIG('contMeas').init_instruments()

lab.HAL("MW_qubit").Power = 16
lab.HAL('MW_qubit').Frequency = lab.SPEC('my_qubit')['Frequency'].Value
lab.HAL("MW_res").Output = False

We will define a new AWG waveform to guide the timing of our experiment. Note that we are outputting a long-time constant pulse, modulated by the `lab.WFMT('QubitFreqGE')` object; this, mixed with the qubit microwave signal, is what we are measuring.

In [ ]:
lab.HAL('DDG').RepetitionTime = 1e-4

stz.WaveformAWG("WfmConMixer", lab, [('awg_ref', 'CH1'), ('awg_ref', 'CH2')], awg_sample_rate)
lab.HAL('WfmConMixer').set_valid_total_time(lab.HAL('DDG').RepetitionTime-1e-6)

total_time = lab.HAL('WfmConMixer').Duration
num_meas_samples = 2**13
meas_time = num_meas_samples / lab.HAL('acq').SampleRate
if meas_time < total_time:
    lab.HAL('acq').set_acq_params(reps = 2**1, segs = 1, samples = num_meas_samples)
else:
    print(f'{num_meas_samples} gives a total measurement time of {meas_time}, which is too long for a waveform of length {total_time}.')

lab.HAL("WfmConMixer").add_waveform_segment(sqdt.WFS_Constant("pad", None, -1, 0.0))
lab.HAL("WfmConMixer").add_waveform_segment(sqdt.WFS_Constant("pulse", lab.WFMT('QubitFreqGE').apply(), meas_time, 0.1))
lab.HAL("WfmConMixer").get_output_channel(0).marker(0).set_markers_to_segments(['pulse'])
lab.HAL("WfmConMixer").set_trigger_source_all(lab.HAL("DDG").get_trigger_output('AB'))
lab.HAL("acq").set_trigger_source(lab.HAL("WfmConMixer").get_output_channel(0).marker(0))
lab.HAL('acq').set_data_processor(myProc)


In this set-up, the acquisition device is being triggered by the AWG to begin acquiring when the `pulse` component of the waveform begins. The variables `meas_time` and `lab.HAL('acq').NumSamples` have been chosen such that the pulse envelope generation and signal acquisition occur for the same period of time. We also re-use the previous data processor, `myProc`.

In [ ]:
stz.ExperimentConfiguration('mixer_calib', lab, lab.HAL('DDG').RepetitionTime, ['DDG', "WfmConMixer", 'MW_dnc', 'MW_qubit', 'MW_res'], 'acq')
lab.CONFIG('mixer_calib').plot()

With this configuration, we should be able to see the qubit frequency signal and the two sidebands, separated by 100 MHz. To check this, you can sweep the downconversion frequency in a range inclusive of 200 MHz in either direction from the qubit frequency signal.

In [ ]:
sqdt.VariableProperty('dnc_freq', lab, lab.HAL("MW_dnc"), 'Frequency')

sqdt.VariableInternal('freq_mixer_calib', lab)
sqdt.VariableSpaced('qubit_freq_mixer_calib', lab, 'freq_mixer_calib', 'dnc_freq', -25e6)

new_exp = sqdt.Experiment('mixer_calib_sweep', lab.CONFIG('mixer_calib'))

centre_freq = lab.HAL('MW_qubit').Frequency

mixer_calib_data = lab.run_single(new_exp, [(lab.VAR('qubit_freq_mixer_calib'), 
            np.arange(-200e6, 200e6, 20e6)+centre_freq)])

This can be visualised in `SQDViz`. After calibrating the mixer, you should come back and perform this measurement again to ensure that only one sideband remains.

### LO mixer calibration

This procedure suppresses the central LO component of the three peaks. This process is thoroughly documented [here](https://github.com/sqdlab/SQDToolz/blob/main/docs/User/Mixer_Calibration.md), so comments will be minimal.

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpMixerCalibrationLO import *

new_exp = ExpMixerCalibrationLO(name = 'mixer_calib_LO', expt_config = lab.CONFIG('mixer_calib'), 
                                var_down_conv_freq = lab.VAR('qubit_freq_mixer_calib'), 
                                freqs_sideband_LCR = [centre_freq-100e6, centre_freq, centre_freq+100e6],
                                lab.VAR('DCoff_I'), (-2,2), lab.VAR('DCoff_Q'), (-2,2))
LO_data = lab.run_single(new_exp)

The ranges for `DCoff_I` and `DCoff_Q` should be adjusted above until a clear bowl-shape is seen. Then, the minimum can be located and we can proceed to autonomous optimisation.

In [ ]:
new_exp = ExpMixerCalibrationLO('mixer_calib_LO', lab.CONFIG('mixer_calib'), lab.VAR('qubit_freq_mixer_calib'), 
                                [centre_freq-30e6, centre_freq, centre_freq+30e6],
                                lab.VAR('DCoff_I'), (-0.5,0.5), lab.VAR('DCoff_Q'), (-0.5,0.5), 
                                optimise=True, sample_points=4, iterations=10, win_shrink_factor=0.33, acq_ch = 'ch1')
LO_data_opt = lab.run_single(new_exp)

The optimisation experiment will leave the variables `DCoff_I` and `DCoff_Q` in the optimal values to suppress the central leakage. Thus, we should save these parameters.

In [ ]:
lab.SPEC(f'MixerCalib')['IdcOff'].Value = lab.VAR('DCoff_I').Value
lab.SPEC(f'MixerCalib')['QdcOff'].Value = lab.VAR('DCoff_Q').Value

By performing the `mixer_calib_sweep` experiment again, it should be clear the central tone is suppressed.

### SB mixer calibration

Sideband mixer calibration, which suppresses one of the sidebands, is extremely similar in execution to LO mixer calibration. Now, we are calibrating the amplitude factor and phase offset mentioned earlier.

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpMixerCalibrationSB import*

upper_SB_freq = centre_freq+100e6

new_exp = ExpMixerCalibrationSB(f'mixer_calib_SB', lab.CONFIG('mixer_calib'), lab.VAR('qubit_freq_mixer_calib'), upper_SB_freq,
                             lab.VAR('IQ_amp_fac'), (0.8,1.05), lab.VAR('IQ_phs_diff'), (2,4))
SB_data = lab.run_single(new_exp)

Again, find search parameters which include a bowl-shaped minimum, and proceed to optimisation.

In [ ]:
new_exp = ExpMixerCalibrationSB('mixer_calib_SB', lab.CONFIG('mixer_calib'), lab.VAR('qubit_freq_mixer_calib'), upper_SB_freq,
                             lab.VAR('IQ_amp_fac'), (0.95,1.05), lab.VAR('IQ_phs_diff'), (3, 3.5), optimise=True, sample_points=4, iterations=10, win_shrink_factor=0.33, acq_ch = 'ch1')
SB_data_opt = lab.run_single(new_exp)

Save the final parameters.

In [ ]:
lab.SPEC(f'MixerCalibQ4')['AmpFac'].Value = lab.WFMT('QubitFreqGE').IQAmplitudeFactor
lab.SPEC(f'MixerCalibQ4')['PhsOff'].Value = lab.WFMT('QubitFreqGE').IQPhaseOffset

A final `mixer_calib_sweep` experiment should confirm only one frequency component (the upper sideband in this case) remains.